### Install Pre-requisites

In [1]:
%%bash
python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


### Import Modules

In [2]:
import json
import math
import random
import time

import spacy
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext.data as td

SEED = 2020

### Create Dataset

In [3]:
!unzip coqa-train-v1.0.zip

Archive:  coqa-train-v1.0.zip
  inflating: coqa-train-v1.0.json    


In [4]:
def iteratively_add_conv_history(psg_with_turns, fields):
    psg = psg_with_turns["story"]
    total_turns = len(psg_with_turns["questions"])
    dataset = []
    for q in range(total_turns):
        question = psg_with_turns["questions"][q]["input_text"]
        answer = psg_with_turns["answers"][q]["input_text"]
        psg += (" chiknichameli " + question)
        dataset.append( (psg, answer, fields) )
        psg += (" chiknachamela " + answer)
    return dataset

def create_dataset_from_raw(alldata, fields):
    dataset = []
    for d in alldata["data"]:
        dataset.extend( iteratively_add_conv_history(psg_with_turns=d, fields=fields) )
    return dataset


spacy_en = spacy.load('en')
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]


SRC = td.Field(sequential=True, tokenize=tokenize_en, init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)
TRG = td.Field(sequential=True, tokenize=tokenize_en, is_target=True, init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)
fields = [("src", SRC), ("trg", TRG)]

alldata = json.load(  open("coqa-train-v1.0.json", "r")  )
example = [  td.Example.fromlist([q,a], f) for q,a,f in create_dataset_from_raw(alldata=alldata, fields=fields)  ] 

ds = td.Dataset(example, fields)

In [5]:
(train_data, valid_data) = ds.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [6]:
SRC.build_vocab(train_data, min_freq = 5)
TRG.build_vocab(train_data, min_freq = 5)

In [7]:
BATCH_SIZE = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def sk(x):
    return len(  next(x.question)  )

# train_iter, valid_iter = td.BucketIterator.splits( (train_data, valid_data), 
#                                                               batch_size=BATCH_SIZE, 
#                                                               device=device )

train_iter = td.BucketIterator( train_data, 
                                                batch_size=BATCH_SIZE, 
                                                device=device )
valid_iter = td.BucketIterator( valid_data, 
                                                batch_size=BATCH_SIZE, 
                                                device=device )

### Create Encoder Model

In [8]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(input_dim, embed_dim)
        self.rnn = nn.GRU(embed_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, inpdata):
        emb = self.dropout(self.embedding(inpdata))
        outputs, hidden = self.rnn(emb)
        return hidden

### Create Decoder Model

In [9]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, embed_dim)
        self.rnn = nn.GRU(  embed_dim + hidden_dim, hidden_dim  )
        self.fc = nn.Linear(  embed_dim + hidden_dim * 2, output_dim  )
        self.dropout = nn.Dropout(dropout)

    def forward(self, inpdata, hidden, context):
        inpdata = inpdata.unsqueeze(0)
        emb = self.dropout(self.embedding(inpdata))
        emb_con = torch.cat([emb, context], dim=2)
        output, hidden = self.rnn(emb_con, hidden)
        output = torch.cat((emb.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        prediction = self.fc(output)
        return prediction, hidden


### Seq To Seq Model

In [10]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hidden_dim == decoder.hidden_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        context = self.encoder(src)
        hidden = context
        input = trg[0,:]
        
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden, context)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1) 
            input = trg[t] if teacher_force else top1

        return outputs

### Initialize model

In [11]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

### Initialize Weights As Given In The Paper

In [12]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(65168, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5985, 256)
    (rnn): GRU(768, 512)
    (fc): Linear(in_features=1280, out_features=5985, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

### Count Parameters

In [13]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 29,033,825 trainable parameters


### Initialize Training Components

In [14]:
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

### Create Train Function

In [15]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        optimizer.zero_grad()
        output = model(src, trg)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

### Create Evaluation Function

In [16]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            output = model(src, trg, 0)
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

### Time Keeper Function

In [17]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### Start Training

In [18]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):    
    start_time = time.time()
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 4m 59s
	Train Loss: 4.859 | Train PPL: 128.957
	 Val. Loss: 4.776 |  Val. PPL: 118.686
Epoch: 02 | Time: 5m 8s
	Train Loss: 4.604 | Train PPL:  99.841
	 Val. Loss: 4.778 |  Val. PPL: 118.921
Epoch: 03 | Time: 5m 12s
	Train Loss: 4.510 | Train PPL:  90.933
	 Val. Loss: 4.776 |  Val. PPL: 118.587
Epoch: 04 | Time: 5m 12s
	Train Loss: 4.455 | Train PPL:  86.013
	 Val. Loss: 4.764 |  Val. PPL: 117.196
Epoch: 05 | Time: 5m 14s
	Train Loss: 4.383 | Train PPL:  80.059
	 Val. Loss: 4.772 |  Val. PPL: 118.120
Epoch: 06 | Time: 5m 13s
	Train Loss: 4.342 | Train PPL:  76.894
	 Val. Loss: 4.780 |  Val. PPL: 119.074
Epoch: 07 | Time: 5m 27s
	Train Loss: 4.290 | Train PPL:  72.958
	 Val. Loss: 4.785 |  Val. PPL: 119.689
Epoch: 08 | Time: 5m 30s
	Train Loss: 4.249 | Train PPL:  70.029
	 Val. Loss: 4.780 |  Val. PPL: 119.122
Epoch: 09 | Time: 5m 26s
	Train Loss: 4.217 | Train PPL:  67.836
	 Val. Loss: 4.778 |  Val. PPL: 118.901
Epoch: 10 | Time: 5m 24s
	Train Loss: 4.152 | Train PPL:

### Reload model and test again

In [19]:
model.load_state_dict(torch.load('tut2-model.pt'))
test_loss = evaluate(model, valid_iter, criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.768 | Test PPL: 117.710 |


### Run the official Evaluation Script
(Check official leaderboard [here](https://stanfordnlp.github.io/coqa/))

In [ ]:
# Create preds file
alldata = json.load(  open("coqa-dev-v1.0.json", "r")  )

def test_iteratively_add_conv_history(psg_with_turns, fields):
    id_and_turns = []
    id = psg_with_turns["id"]
    psg = psg_with_turns["story"]
    total_turns = len(psg_with_turns["questions"])
    dataset = []
    turn_num = 1
    for q in range(total_turns):
        id_and_turns.append( (id, turn_num) )
        question = psg_with_turns["questions"][q]["input_text"]
        answer = psg_with_turns["answers"][q]["input_text"]
        psg += (" chiknichameli " + question)
        dataset.append( (psg, answer, fields) )
        psg += (" chiknachamela " + answer)
        turn_num += 1
    return dataset, id_and_turns

def create_dataset_from_raw_test(alldata, fields):
    ids_and_turns = []
    dataset = []
    for d in alldata["data"]:
        recursively_added_data, iat = test_iteratively_add_conv_history(psg_with_turns=d, fields=fields)
        dataset.extend( recursively_added_data )
        ids_and_turns.extend( iat )
    return dataset, ids_and_turns


example_ds, ids_and_turns  = create_dataset_from_raw_test(alldata=alldata, fields=fields)
example = [  td.Example.fromlist([q,a], f) for q,a,f in  example_ds ] 

test_ds = td.Dataset(example, fields)

SRC.build_vocab(test_ds, min_freq = 5)
TRG.build_vocab(test_ds, min_freq = 5)

# test_iter = td.BucketIterator( test_ds, 
#                                                 batch_size=BATCH_SIZE, 
#                                                 device=device )

In [21]:
# !python evaluate-v1.0.py --data-file "coqa-train-v1.0.json" --pred-file <path_to_predictions>